#### Load libraries

In [1]:
# standard libraries
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# from our code-base
import pac_explanation
from data.objects import zoo, iris, adult
from pac_explanation import example_queries
from pac_explanation.blackbox import *
from pac_explanation.learner import *
from pac_explanation.query import *
from pac_explanation.sygus_if import *
from pac_explanation.teacher import *

### Load dataset

Extra information that we need are following.

(1) `attribute_names` : feature names of the dataset,

(2) `attribute_type`: a map between an attribute and its data type; we consider either "Bool" or "Real" (ordinal or numerical)attributes where categorical values have to be one-hot encoded (thus "Bool"),

(3) `real_attribute_domain_info` : for real-valued attributes, we construct a map where the key is the attribute and the value is a tuple (max_value, min_value) for that attribute. In case `real_attribute_domain_info` is incomplete or not provided, we assume that the attribute values are within \[0,1\]

For example, see [zoo dataset](data/objects/zoo.py)



In [2]:
# choose one dataset
dataset = ['zoo', 'adult', 'iris'][1]

df = None

if(dataset == "zoo"):
    dataObj = zoo.Zoo()
    df = dataObj.get_df()
    # fix target class because zoo dataset has 7 classes while we support binary class datasets in this implementation
    target_class = [4] 
    _temp = {}
    for i in range(1, len(df[dataObj.target].unique())+1):
        if(i in target_class):
            _temp[i] = 1
        else:
            _temp[i] = 0
    df[dataObj.target] = df[dataObj.target].map(_temp)
elif(dataset == "adult"):
    dataObj = adult.Adult() 
    df = dataObj.get_df()
elif(dataset == "iris"):
    dataObj = iris.Iris()
    df = dataObj.get_df()

df

Calling one hot encoder on ['race', 'sex', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country']
workclass  is not considered in classification
education  is not considered in classification
marital-status  is not considered in classification
occupation  is not considered in classification
relationship  is not considered in classification
native-country  is not considered in classification


,sex,age,education-num,capital-gain,capital-loss,hours-per-week,target,race_0,race_1,race_2,race_3,race_4
0,1,0.301370,0.800000,0.021740,0.0,0.397959,1,0,0,0,0,1
1,1,0.452055,0.800000,0.000000,0.0,0.122449,1,0,0,0,0,1
2,1,0.287671,0.533333,0.000000,0.0,0.397959,1,0,0,0,0,1
3,1,0.493151,0.400000,0.000000,0.0,0.397959,1,0,0,1,0,0
4,0,0.150685,0.800000,0.000000,0.0,0.397959,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0,0.301370,0.800000,0.000000,0.0,0.357143,1,0,0,0,0,1
48838,1,0.643836,0.533333,0.000000,0.0,0.397959,1,0,0,1,0,0
48839,1,0.287671,0.800000,0.000000,0.0,0.500000,1,0,0,0,0,1
48840,1,0.369863,0.800000,0.054551,0.0,0.397959,1,0,1,0,0,0


#### Dataset preparation for training

In [3]:
# declaration of classifier, X and y
X = df.drop([dataObj.target], axis=1)
y = df[dataObj.target]

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle = True, random_state=2) # 70% training and 30% test

### Train a black-box model

In [4]:
# Preload/store trained model

select_blackbox = ['decision tree','neural network', 'random forest'][1]

model_name = None
if(select_blackbox == 'decision tree'):
    model_name = 'data/model/dt_' + dataset + '.pkl'
elif(select_blackbox == "random forest"):
    model_name = 'data/model/rf_' + dataset + '.pkl'
elif(select_blackbox == "neural network"):
    model_name = 'data/model/nn_' + dataset + '.pkl'

else:
    raise ValueError("Black box not defined")



if(not os.path.isfile(model_name)):
    clf = None
    if(select_blackbox == 'decision tree'):
        param_grid = {'max_depth': np.arange(3, 10)}
        grid_tree = GridSearchCV(tree.DecisionTreeClassifier(random_state=0), param_grid)
        grid_tree.fit(X_train, y_train)
        tree_preds = grid_tree.predict_proba(X_test)[:, 1]
        tree_performance = roc_auc_score(y_test, tree_preds)
        clf = grid_tree.best_estimator_
        print(helper_functions.tree_to_code(clf,X_train.columns.tolist()))
    elif(select_blackbox == "random forest"):
        clf = RandomForestClassifier(n_estimators=100)
        clf.fit(X_train,y_train)

    elif(select_blackbox == "neural network"):
        clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train) 
        clf.fit(X_train,y_train)
        
    else:
        raise ValueError("Black box not defined")

    # store the classifier
    with open(model_name, 'wb') as fid:
        pickle.dump(clf, fid)    

else:
    print("Loding model")
    with open(model_name, 'rb') as fid:
        clf = pickle.load(fid)

    
    # For visualization
    if(select_blackbox == "decision tree"):
        print(pac_explanation.utils.tree_to_code(clf,X_train.columns.tolist()))



Loding model


## Specify target sample (queries with distance parameter)
In this experiment, we have considered two types of queries: 
    
    (1) target sample,
    (2) a logical specification of the sample space viewed as a region    
    
for which, we learn a local explanation

In [5]:
select_query = ['decision tree', 'specific input'][1]

# X and y will be used as arguments for learner as initial examples. While this is not a strict requirement, we make sure that the learner is atleast correct with respect to the given sample (when select_query = specific_input).

X = None
y = None 

# Our PAC explainer considers a query object. For either type of queries, we define a query_class
query_class = None
if(select_query == "decision tree"):
    # We define query specilized for decision tree
    query_class = example_queries.DecisionTree(features=X_train.columns.tolist(), halfspace=example_queries.ExampleQueries().queries[0])
    X = []
    y = []
elif(select_query == "specific input"):  
    # An input from the training set
    specific_input = X_train.iloc[0].tolist()

    # A threshold is provided for distance based query. The range of this threshold is not properly checked. Roughly, all neighboring samples with distance less than the threshold are considered inside the local region
    query_class = example_queries.DistanceQuery(specific_input=specific_input, threshold=0.5, features = X_train.columns.tolist())
    X = [specific_input]
    y = [clf.predict([specific_input])[0]]
    print(query_class)
    print("Label of this sample by the black box predictor:", y)
    
else:
    raise ValueError(select_query +" is not a defined query.")

Query is a specific input
- threshold: 0.5
- specific_input: [1.0, 0.2465753424657534, 0.5333333333333333, 0.0, 0.0, 0.39795918367346933, 0.0, 0.0, 0.0, 0.0, 1.0]
- detailed_input: [('sex', 1.0), ('age', 0.2465753424657534), ('education-num', 0.5333333333333333), ('capital-gain', 0.0), ('capital-loss', 0.0), ('hours-per-week', 0.39795918367346933), ('race_0', 0.0), ('race_1', 0.0), ('race_2', 0.0), ('race_3', 0.0), ('race_4', 1.0)]
Label of this sample by the black box predictor: [1]


## Generate a PAC explanation with SyGuS
We define three entities here.

    (1) blackbox,
    (2) learner, and
    (3) query

Our task is to define a `teacher` who performs the task of learning an explanation of the `blackbox` given the `query` by sending necessary information to the `learner`.

In [6]:
# define the black-box as a scikit-learn object (trained)
blackbox = None
if(select_blackbox == 'decision tree' or select_blackbox == "random forest" or select_blackbox == "neural network"):
    blackbox = BlackBox(clf, clf.predict)
else:
    raise ValueError("Black box not defined")

# define the learner as a SyGuS object
sygus = SyGuS_IF(rule_type="CNF", feature_names=dataObj.attributes, feature_data_type=dataObj.attribute_type, function_return_type= "Bool", real_attribute_domain_info=dataObj.real_attribute_domain_info, verbose=False, syntactic_grammar=True, workdir="temp")

learner = Learner(model = sygus, prediction_function = sygus.predict_z3, train_function = sygus.fit, X = X, y=y)

# define the query as specified above
query = Query(model = None, prediction_function = query_class.predict_function_query)


### Teacher

In [11]:
# Now let the teacher complete its task
teacher = Teacher(max_iterations=100000, epsilon=0.051, delta=0.501, timeout=10)

# Teacher object needs an access to a random sample generator for the verification task. We call a specilized random generator that utilizes the knowledge of the training dataset, for example, the attribute type of each attribute.

teach_start = time.time()
learner, flag = teacher.teach(blackbox = blackbox, learner = learner, query = query, random_example_generator = utils.random_generator, params_generator = (X_train, dataObj.attribute_type), verbose=False)
teach_end = time.time()

print("Explanation for =>")
print(query_class)
print("\nLearned explanation =>")
print(learner.model._function_snippet)
print("\n-explanation size:", learner.model.get_formula_size(verbose=False))
print("-is explanation learning complete?", flag)
print("-it took", teach_end - teach_start, "seconds")
print("-learner time:", teacher.time_learner)
print("-verifier time:", teacher.time_verifier)
print('-random words checked', teacher.verifier.number_of_examples_checked)
print("-random words filtered by querys:", teacher.verifier.filtered_by_query)
print("-total counterexamples:", len(learner.y))
print("-percentage of positive counterexamples for the learner:", np.array(learner.y).mean())


Explanation for =>
Query is a specific input
- threshold: 0.5
- specific_input: [1.0, 0.2465753424657534, 0.5333333333333333, 0.0, 0.0, 0.39795918367346933, 0.0, 0.0, 0.0, 0.0, 1.0]
- detailed_input: [('sex', 1.0), ('age', 0.2465753424657534), ('education-num', 0.5333333333333333), ('capital-gain', 0.0), ('capital-loss', 0.0), ('hours-per-week', 0.39795918367346933), ('race_0', 0.0), ('race_1', 0.0), ('race_2', 0.0), ('race_3', 0.0), ('race_4', 1.0)]

Learned explanation =>
 (and (< capital-loss (/ 4 5)) (or (< education-num (/ 2 5)) (or (> education-num (/ 4 5)) (< hours-per-week (/ 39999999999999997 100000000000000000)))))

-explanation size: 5
-is explanation learning complete? False
-it took 11.009070873260498 seconds
-learner time: 9.944893836975098
-verifier time: 0.06206011772155762
-random words checked 1
-random words filtered by querys: 0
-total counterexamples: 14
-percentage of positive counterexamples for the learner: 0.35714285714285715


#### Evaluation of the explanation

In [9]:
# If a test sample is not within the query, we do not report explanation error, otherwise we report error when the learner and the blackbox have different classification for that sample

acc = None
try:
    cnt = 0
    learner_verdicts = learner.classify_examples(X_test.values.tolist())
    blackbox_verdicts = blackbox.classify_examples(X_test.values.tolist())
    for i in range(len(X_test.values.tolist())):

        blackbox_verdict = blackbox_verdicts[i]
        learner_verdict = learner_verdicts[i]
        query_verdict = query.classify_example(X_test.values.tolist()[i])
        if(not query_verdict):
            cnt += 1
            continue
        if(learner_verdict == blackbox_verdict):
            cnt += 1
    acc = cnt/len(y_test)
except:
    cnt = None
    acc = None

print("correct: ", cnt, "out of ", len(y_test), "examples. Percentage: ", acc)

-is explanation learning complete? False
-it took 11.018446922302246 seconds
-learner time: 9.932276964187622
-verifier time: 0.0792238712310791
correct:  3845 out of  4885 examples. Percentage:  0.7871033776867963
random words checked 1
Filtered by querys: 0
Total counterexamples: 12
percentage of positive counterexamples for the learner: 0.4166666666666667

